In [2]:
import cartopy.io.shapereader as shpreader
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import pandas as pd
import os
import sys
from osgeo import gdal,ogr, osr
import xarray
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

Soilgrids250m: SRTM Digital Elevation Data Version 4
* https://gee-community-catalog.org/projects/isric/
* https://gee-community-catalog.org/projects/polaris/

In [5]:
## Mosaic data
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/9_soil_data/1_original_data/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/9_soil_data/2_mosaic_data/"
file_lists = os.listdir(data_path)

variables = ["bdod", "cec", "cfvo","clay","nitrogen","ocd","ocs","phh20","sand","silt","soc","theta_r","theta_s","ksat"]

for var in variables:
    data_list = [f"{data_path}{x}" for x in file_lists if var in x and ".tif" in x]
    out_tif = f"{out_path}{var}.tif"
    
    options = gdal.WarpOptions(format='GTiff')
    mosaic = gdal.Warp(out_tif, data_list, options=options)
    mosaic = None 
    print("Mosaic created successfully at:", var)

Mosaic created successfully at: bdod
Mosaic created successfully at: cec
Mosaic created successfully at: cfvo
Mosaic created successfully at: clay
Mosaic created successfully at: nitrogen
Mosaic created successfully at: ocd
Mosaic created successfully at: ocs
Mosaic created successfully at: phh20
Mosaic created successfully at: sand
Mosaic created successfully at: silt
Mosaic created successfully at: soc
Mosaic created successfully at: theta_r
Mosaic created successfully at: theta_s
Mosaic created successfully at: ksat


In [13]:
## convert to WGS-84
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/9_soil_data/2_mosaic_data/"
variables = ["bdod", "cec", "cfvo","clay","nitrogen","ocd","ocs","phh20","sand","silt","soc"]
for var in variables:
    input_tif = f"{data_path}{var}.tif"
    out_tif = f"{data_path}{var}.tif"
    target_srs = 'EPSG:4326'
    gdal.Warp(out_tif, input_tif, dstSRS=target_srs)
    out_tif = None
    input_tif = None
    print(var,datetime.now().strftime("%Y-%m-%d %H:%M:%S"),"finished")

bdod 2024-07-12 14:24:13 finished
cec 2024-07-12 14:25:00 finished
cfvo 2024-07-12 14:25:48 finished
clay 2024-07-12 14:26:37 finished
nitrogen 2024-07-12 14:27:25 finished
ocd 2024-07-12 14:28:13 finished
ocs 2024-07-12 14:29:02 finished
phh20 2024-07-12 14:29:50 finished
sand 2024-07-12 14:30:38 finished
silt 2024-07-12 14:31:26 finished
soc 2024-07-12 14:32:15 finished


In [14]:
## clip
def read_tif(tif_file):
    dataset = gdal.Open(tif_file)
    cols = dataset.RasterXSize
    rows = dataset.RasterYSize
    im_proj = (dataset.GetProjection())
    im_Geotrans = (dataset.GetGeoTransform())
    im_data = np.moveaxis(dataset.ReadAsArray(0, 0, cols, rows), 0, -1)
    return im_data, im_Geotrans, im_proj,rows, cols

prisma_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/2_PRISMA_L2D/2_PRISMA_L2D_tif_2020_2023/"
data_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/9_soil_data/2_mosaic_data/"
out_path = "/Volumes/ChenLab/Fujiang/0_PhD_dissertation_data/9_soil_data/3_clipped_NEON_extent/"

variables = ["bdod", "cec", "cfvo","clay","nitrogen","ocd","ocs","phh20","sand","silt","soc","theta_r","theta_s","ksat"]
site_list = ['D01_BART','D01_HARV','D02_SCBI','D03_OSBS','D07_MLBS','D07_ORNL','D08_TALL',
             'D10_CPER','D13_MOAB','D14_JORN','D14_SRER','D16_WREF','D19_BONA','D19_HEAL']

for site in site_list:
    path = f"{prisma_path}{site}"
    file_list = os.listdir(path)
    file_list = [x for x in file_list if "LATLON.tif" in x]

    ul_x_all,lr_y_all, lr_x_all, ul_y_all = [],[],[],[]
    for file in file_list:
        im_data, im_Geotrans, im_proj,rows, cols = read_tif(f"{path}/{file}")
        ul_x, lr_y, lr_x, ul_y = im_data[0,0,1],im_data[-1,-1,0],im_data[-1,-1,1],im_data[0,0,0]
        ul_x_all.append(ul_x)
        lr_y_all.append(lr_y)
        lr_x_all.append(lr_x)
        ul_y_all.append(ul_y)
    ul_x, lr_y, lr_x, ul_y = min(ul_x_all),min(lr_y_all), max(lr_x_all), max(ul_y_all)
    ul_x, lr_y, lr_x, ul_y = ul_x-0.2, lr_y-0.2, lr_x+0.2, ul_y+0.2
    print(site, ul_x, lr_y, lr_x, ul_y)
    
    os.makedirs(f"{out_path}{site}", exist_ok=True)
    for var in variables:
        file = f"{var}.tif"
        input_tif = f"{data_path}{file}"
        output_tif =f"{out_path}{site}/{file}"
        gdal.Warp(output_tif, input_tif, format = 'GTiff', outputBounds=(ul_x, lr_y, lr_x, ul_y))
        input_tif = None
        output_tif = None
        print(site,var,datetime.now().strftime("%Y-%m-%d %H:%M:%S"),"finished")
        
        ### transfer the projection of clipped data
        in_tif = f"{out_path}{site}/{file}"

        out_tif = f"{out_path}{site}/{file}"
        input_ds = gdal.Open(in_tif)
        output_ds = gdal.Warp(out_tif, input_ds, dstSRS=im_proj)
        input_ds = None
        output_ds = None

D01_BART -71.75350494384766 43.562035369873044 -70.66015319824218 44.545073699951175
D01_BART bdod 2024-07-12 14:33:05 finished
D01_BART cec 2024-07-12 14:33:05 finished
D01_BART cfvo 2024-07-12 14:33:05 finished
D01_BART clay 2024-07-12 14:33:05 finished
D01_BART nitrogen 2024-07-12 14:33:05 finished
D01_BART ocd 2024-07-12 14:33:05 finished
D01_BART ocs 2024-07-12 14:33:05 finished
D01_BART phh20 2024-07-12 14:33:05 finished
D01_BART sand 2024-07-12 14:33:05 finished
D01_BART silt 2024-07-12 14:33:05 finished
D01_BART soc 2024-07-12 14:33:05 finished
D01_BART theta_r 2024-07-12 14:33:37 finished
D01_BART theta_s 2024-07-12 14:34:10 finished
D01_BART ksat 2024-07-12 14:34:44 finished
D01_HARV -72.74523468017578 42.03336029052734 -71.72100524902343 43.04200286865235
D01_HARV bdod 2024-07-12 14:34:52 finished
D01_HARV cec 2024-07-12 14:34:52 finished
D01_HARV cfvo 2024-07-12 14:34:53 finished
D01_HARV clay 2024-07-12 14:34:53 finished
D01_HARV nitrogen 2024-07-12 14:34:53 finished
D01_H